
# Problem 2 — **Simple** Notebook
Inputs:
1. `2B69_pot97129.rawiv` (electrostatic potential, **RAWIV big-endian**)
2. `isosurface_vertices.raw` (binary **RAW** of packed `(x,y,z)` as `float32` — endianness auto-detected)

Outputs:
- Min/Mean/Max of potentials
- CSV with columns: `x,y,z,potential`


In [ ]:

# === Minimal helpers: RAWIV reader (big-endian) + RAW vertex loader ===
import struct, numpy as np
from pathlib import Path
from dataclasses import dataclass

@dataclass
class RawIVHeader:
    minX: float; minY: float; minZ: float
    maxX: float; maxY: float; maxZ: float
    numVerts: int; numCells: int
    dimX: int; dimY: int; dimZ: int
    originX: float; originY: float; originZ: float
    spanX: float; spanY: float; spanZ: float

def read_rawiv_header(path):
    with open(path, "rb") as f:
        b = f.read(68)
    vals = struct.unpack(">6f2I3I3f3f", b)
    return RawIVHeader(*vals)

def read_rawiv_data(path, dtype="float32"):
    hdr = read_rawiv_header(path)
    dt = {"float32": ">f4", "uint16": ">u2", "uint8": ">u1"}[dtype]
    count = hdr.dimX * hdr.dimY * hdr.dimZ
    with open(path, "rb") as f:
        f.seek(68)
        data = np.fromfile(f, dtype=np.dtype(dt), count=count)
    vol = data.reshape((hdr.dimZ, hdr.dimY, hdr.dimX))
    return hdr, vol

def load_vertices_raw_auto(path, hdr):
    """Load (x,y,z) float32 from .raw.
    Auto-detect endianness by checking how many points fall inside the RAWIV bbox.
    """
    path = Path(path)
    size = path.stat().st_size
    if size % 12 != 0:
        raise ValueError(f"File size {size} is not a multiple of 12 bytes (float32 xyz)." )
    n = size // 12
    with open(path, "rb") as f:
        raw = f.read()
    arr_be = np.frombuffer(raw, dtype=">f4").reshape(-1,3)
    arr_le = np.frombuffer(raw, dtype="<f4").reshape(-1,3)
    # count how many points lie within bbox for each interpretation
    def in_bbox(a):
        return np.sum((a[:,0]>=hdr.minX)&(a[:,0]<=hdr.maxX)&
                      (a[:,1]>=hdr.minY)&(a[:,1]<=hdr.maxY)&
                      (a[:,2]>=hdr.minZ)&(a[:,2]<=hdr.maxZ))
    c_be, c_le = in_bbox(arr_be), in_bbox(arr_le)
    if c_be >= c_le and c_be>0:
        return arr_be
    elif c_le>0:
        return arr_le
    # fallback: assume big-endian
    return arr_be


In [ ]:

# === Trilinear interpolation (minimal) ===
import numpy as np

def world_to_grid(xyz, hdr):
    fx = (xyz[:,0] - hdr.minX) / hdr.spanX
    fy = (xyz[:,1] - hdr.minY) / hdr.spanY
    fz = (xyz[:,2] - hdr.minZ) / hdr.spanZ
    ix, iy, iz = np.floor(fx).astype(int), np.floor(fy).astype(int), np.floor(fz).astype(int)
    ix = np.clip(ix, 0, hdr.dimX-2); iy = np.clip(iy, 0, hdr.dimY-2); iz = np.clip(iz, 0, hdr.dimZ-2)
    u, v, w = fx - ix, fy - iy, fz - iz
    return ix, iy, iz, u, v, w

def trilinear(vol, hdr, xyz):
    ix, iy, iz, u, v, w = world_to_grid(xyz, hdr)
    f000 = vol[iz,     iy,     ix    ]; f100 = vol[iz,     iy,     ix+1]
    f010 = vol[iz,     iy+1,   ix    ]; f110 = vol[iz,     iy+1,   ix+1]
    f001 = vol[iz+1,   iy,     ix    ]; f101 = vol[iz+1,   iy,     ix+1]
    f011 = vol[iz+1,   iy+1,   ix    ]; f111 = vol[iz+1,   iy+1,   ix+1]
    U, V, W = u, v, w
    return (
        f000*(1-U)*(1-V)*(1-W) + f100*U*(1-V)*(1-W) +
        f010*(1-U)*V*(1-W)     + f110*U*V*(1-W)     +
        f001*(1-U)*(1-V)*W     + f101*U*(1-V)*W     +
        f011*(1-U)*V*W         + f111*U*V*W
    ).astype(float)


In [ ]:

# === EDIT ONLY THESE PATHS ===
pot_rawiv = r"C:\path\to\2B69_pot97129.rawiv"
verts_raw  = r"C:\path\to\isosurface_vertices.raw"   # binary RAW of float32 xyz

# === RUN ===
hdr, vol = read_rawiv_data(pot_rawiv, dtype="float32")
V = load_vertices_raw_auto(verts_raw, hdr)

vals = trilinear(vol, hdr, V)

print(f"Vertices: {len(V):,}")
print(f"Potential min/mean/max: {vals.min():.6f}  {vals.mean():.6f}  {vals.max():.6f}")

# Save CSV next to the RAW vertices
out_csv = Path(verts_raw).with_suffix(".potentials.csv")
np.savetxt(out_csv, np.column_stack([V, vals]), delimiter=",", header="x,y,z,potential", comments="")
print(f"Saved: {out_csv}")
